## Competição 3 - DSWA 2025/2
### Desenvolvido por Luis Felipe RA - 813539 e Felipe Bonadia RA - 813908

In [ ]:
### %% [code] {"execution":{"iopub.status.busy":"2025-12-30T17:23:16.657391Z","iopub.execute_input":"2025-12-30T17:23:16.658150Z","iopub.status.idle":"2025-12-30T17:23:16.671924Z","shell.execute_reply.started":"2025-12-30T17:23:16.658119Z","shell.execute_reply":"2025-12-30T17:23:16.671149Z"}}
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importações

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install seaborn

import xgboost as xgb
from sklearn.metrics import mean_squared_error
import holidays
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    ExtraTreesRegressor
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

## Análise exploratória e pré processamento

### Carregando os dados

In [ ]:
df_train = pd.read_csv('/kaggle/input/visagio-previsao-de-demanda-diaria/train.csv')
df_train.columns = [f'feature{i+1}' for i in range(len(df_train.columns))]
df_train = df_train.rename(columns={'feature6': 'target'})
df_produto = pd.read_csv('/kaggle/input/visagio-previsao-de-demanda-diaria/produto.csv')
df_produto.columns = [f'feature{i+1}' for i in range(len(df_train.columns), len(df_train.columns)+len(df_produto.columns))]
df = pd.merge(df_train, df_produto, left_on='feature2', right_on='feature10', how='left').drop(columns=['feature10'])

### Verificação de dados nulos

In [ ]:
df.isnull().sum()

### Transformações

In [ ]:
lefeature4 = LabelEncoder()
df['feature4'] = lefeature4.fit_transform(df['feature4'])
lefeature5 = LabelEncoder()
df['feature5'] = lefeature5.fit_transform(df['feature5'])
lefeature12 = LabelEncoder()
df['feature12'] = lefeature12.fit_transform(df['feature12'])
lefeature13 = LabelEncoder()
df['feature13'] = lefeature13.fit_transform(df['feature13'])
df['feature1'] = pd.to_datetime(df['feature1'])
df['feature14'] = df['feature1'].dt.year
df['feature15'] = df['feature1'].dt.month
df['feature16'] = df['feature1'].dt.day

holidays_br = holidays.Brazil()
df['feature17'] = df['feature1'].apply(lambda x: 1 if x in holidays_br else 0)
df['feature18'] = df['feature1'].dt.dayofweek.isin([5, 6]).astype(int)
df['feature19'] = df['feature1'].dt.is_month_start.astype(int)
df['feature20'] = df['feature1'].dt.is_month_end.astype(int)

df = df.drop(columns=['feature11'])

### Correlação entre variaveis de entrada e coluna alvo

In [ ]:
df_corr = df.drop(columns=['feature1'])
colunas_alvo = [col for col in df_corr.columns if col.startswith('target')]
colunas_entrada = [col for col in df_corr.columns if col.startswith('feature')]
corr = df_corr.corr()
corr_entrada_alvo = corr.loc[colunas_entrada, colunas_alvo]
plt.figure(figsize=(12, 8))
sns.heatmap(corr_entrada_alvo, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlação entre variáveis de entrada e colunas-alvo")
plt.show()

### Escolhendo as variáveis mais relevantes

In [ ]:
#df = df[['feature2', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13', 'target']]
#df = df[['feature2', 'feature3', 'feature4', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13',  'feature15', 'feature16', 'feature17', 'feature18', 'target']]
#df = df[['feature2', 'feature3', 'feature4', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13', 'feature15', 'feature18', 'feature21', 'feature22', 'feature23', 'target']]
#df = df[['feature2', 'feature3', 'feature4', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13', 'feature15', 'feature18', 'target']]

df_final = df[['feature2', 'feature3', 'feature4', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13', 'feature15', 'feature18', 'target']]

### Separação em features e targets

In [ ]:
dffeatures = df_final.drop(columns=['target'])
dftarget = df_final.drop(columns=[col for col in df_final.columns if col.startswith('feature')])

### Divisão em treino e teste

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dffeatures, dftarget, test_size=0.2, random_state=42)

## Testando modelos

In [ ]:
import optuna
from xgboost import XGBRegressor

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "random_state": 42,
        "n_jobs": -1
    }

    model = XGBRegressor(**params)
    model.fit(x_train, y_train.values.ravel())

    preds = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)

print("Melhores parâmetros:", study.best_params)
print("Melhor RMSE:", study.best_value)


In [ ]:
melhor_modelo = XGBRegressor(
    **study.best_params,
    random_state=42,
    n_jobs=-1
)

melhor_modelo.fit(dffeatures, dftarget.values.ravel())
melhor_nome = "XGBoost + Optuna"


In [ ]:
modelos = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=42),
    "KNN": KNeighborsRegressor(),
    "SVR": SVR(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "MLP": MLPRegressor(max_iter=1000, random_state=42)
}

resultados = []
melhor_modelo = None
melhor_rmse = float('inf')
melhor_nome = ""

for nome, modelo in modelos.items():
    modelo.fit(x_train, y_train.values.ravel())
    
    y_pred = modelo.predict(x_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    resultados.append((nome, rmse))

    if rmse < melhor_rmse:
        melhor_rmse = rmse
        melhor_modelo = modelo
        melhor_nome = nome

resultados.sort(key=lambda x: x[1])
df_resultados = pd.DataFrame(resultados, columns=['Modelo', 'RMSE'])

print("\nResultados dos testes:")
print(df_resultados)
print(f"\nMelhor modelo: {melhor_nome} com RMSE de {melhor_rmse:.4f}")

In [ ]:
# Criar ensemble dos 3 melhores modelos
from sklearn.ensemble import VotingRegressor

# Pegar os 3 melhores modelos da lista de resultados
top_3_nomes = [nome for nome, _ in resultados[:3]]

print(f"\nCriando ensemble com: {top_3_nomes}")

# Criar ensemble
ensemble_models = [(nome, modelos[nome]) for nome in top_3_nomes]
ensemble = VotingRegressor(estimators=ensemble_models)

# Treinar ensemble
ensemble.fit(x_train, y_train.values.ravel())

# Avaliar ensemble
y_pred_ensemble = ensemble.predict(x_test)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))

print(f"RMSE do Ensemble: {rmse_ensemble:.4f}")

# Se o ensemble for melhor, usar ele
if rmse_ensemble < melhor_rmse:
    melhor_modelo = ensemble
    melhor_nome = "Ensemble"
    melhor_rmse = rmse_ensemble
    print(f"✅ Ensemble é o melhor modelo!")

In [ ]:
df_test_raw = pd.read_csv('/kaggle/input/visagio-previsao-de-demanda-diaria/test.csv')
df_produto_raw = pd.read_csv('/kaggle/input/visagio-previsao-de-demanda-diaria/produto.csv')
df_test_raw.columns = [f'feature{i+1}' for i in range(len(df_test_raw.columns))]
ids_originais = df_test_raw['feature1']
df_produto_raw.columns = [f'feature{i+1}' for i in range(len(df_test_raw.columns), len(df_test_raw.columns)+len(df_produto_raw.columns))]
df_test = pd.merge(df_test_raw, df_produto_raw, left_on='feature3', right_on='feature7', how='left').drop(columns=['feature7'])
df_test = df_test.drop(columns=['feature1', 'feature8'])
df_test = df_test.rename(columns={'feature2':'feature1', 'feature3':'feature2', 'feature4':'feature3', 'feature5':'feature4', 'feature6':'feature5', 'feature9':'feature12', 'feature10': 'feature13'})
df_test['feature4'] = lefeature4.transform(df_test['feature4'].astype(str).str.strip())
df_test['feature5'] = lefeature5.transform(df_test['feature5'])
df_test['feature12'] = lefeature12.transform(df_test['feature12'])
df_test['feature13'] = lefeature13.transform(df_test['feature13'])
df_test['feature1'] = pd.to_datetime(df_test['feature1'])
df_test['feature14'] = df_test['feature1'].dt.year
df_test['feature15'] = df_test['feature1'].dt.month
df_test['feature16'] = df_test['feature1'].dt.day

holidays_br = holidays.Brazil()
df_test['feature17'] = df_test['feature1'].apply(lambda x: 1 if x in holidays_br else 0)
df_test['feature18'] = df_test['feature1'].dt.dayofweek.isin([5, 6]).astype(int)
df_test['feature19'] = df_test['feature1'].dt.is_month_start.astype(int)
df_test['feature20'] = df_test['feature1'].dt.is_month_end.astype(int)

df_test['feature7'] = np.nan
df_test['feature8'] = np.nan
df_test['feature9'] = np.nan

for index, row in df_test.iterrows():
    filter_value = row['feature1'].weekday()
    
    subset = df[df['feature1'].dt.weekday == filter_value]

    modefeature7 = subset['feature7'].mean()
    modefeature8 = subset['feature8'].mean()
    modefeature9 = subset['feature9'].mean()
   
    if not np.isnan(modefeature7):
     df_test.at[index, 'feature7'] = modefeature7

    if not np.isnan(modefeature8):
     df_test.at[index, 'feature8'] = modefeature8

    if not np.isnan(modefeature9):
     df_test.at[index, 'feature9'] = modefeature9

df_test = df_test.drop(columns=['feature1'])
df_test_final = df_test[['feature2', 'feature3', 'feature4', 'feature5', 'feature7', 'feature8', 'feature9', 'feature12', 'feature13', 'feature15', 'feature18']]

y_submission = melhor_modelo.predict(df_test_final)

submission = pd.DataFrame({
    "id": ids_originais,
    "demanda": y_submission
})

submission.to_csv("submission.csv", index=False)
print(f"Sucesso! Arquivo 'submission.csv' gerado usando {melhor_nome}.")